In [1]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import acquire
import prepare

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, MinMaxScaler

In [2]:
# Cleaned Data for Exploration
df = acquire.get_telco_data(cached = False)
train, validate, test = prepare.prep_telco_data(df)

In [3]:
train

,senior_citizen,partner,dependents,tenure,phone_service,paperless_billing,monthly_charges,total_charges,churn,contract_type_id.1,...,yes_streaming_movies,month_to_month_contract,one_year_contract,two_year_contract,dsl,fiber_optic,auto_bank_transfer,auto_credit_card,e_check,mail_check
5670,0,0,1,5,1,0,20.35,122.00,0,1,...,0,1,0,0,0,0,0,1,0,0
4152,0,0,0,6,1,1,89.25,487.05,0,1,...,1,1,0,0,0,1,0,0,1,0
4890,0,0,0,54,1,0,84.40,4484.05,0,2,...,0,0,1,0,0,1,0,0,1,0
2391,0,1,0,72,1,0,64.75,4804.75,0,3,...,0,0,0,1,1,0,1,0,0,0
2896,0,1,0,28,1,0,76.55,2065.40,0,1,...,0,1,0,0,0,1,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4758,0,1,0,66,1,1,104.05,6890.00,1,2,...,1,0,1,0,0,1,0,0,1,0
381,1,1,0,51,0,1,29.45,1459.35,0,1,...,0,1,0,0,1,0,0,1,0,0
2622,0,0,0,7,1,0,74.65,544.55,1,1,...,0,1,0,0,0,1,0,0,0,1
3812,0,1,0,60,1,1,106.15,6411.25,0,1,...,1,1,0,0,0,1,0,0,1,0


In [ ]:
#titanic = acquire.get_titanic_data(cached=True)

In [ ]:
#train, validate, test = prepare.prep_titanic(titanic)

In [ ]:
#train.info()

In [ ]:
#validate.info()

In [ ]:
#test.info()

# Messing around with titanic data for exploratory data analysis

In [ ]:
df = acquire.get_titanic_data(cached=True)
train, validate, test = prepare.prep_titanic(df)

train = train[['survived', 'pclass', 'sex', 'age', 'fare', 'embark_town', 'alone']]

train.head()

In [ ]:
survival_rate = train.survived.mean()

In [ ]:
train.survived.value_counts().plot.bar()
plt.xlabel('Survived')

In [ ]:
train.groupby('sex').survived.mean().plot.bar(alpha=.8)
plt.ylabel('Survival Rate')
plt.hlines(survival_rate, *plt.xlim(), ls='--', alpha=.8)

In [ ]:
train.groupby('alone').survived.mean().plot.bar(alpha=.8)
plt.ylabel('Survival Rate')
plt.hlines(survival_rate, *plt.xlim(), ls='--', alpha=.8)

In [ ]:
# Exploring 2 categorical variables, but now we're treating survived as a category, not a number.
pd.crosstab(train.survived, train.sex)

In [ ]:
pd.crosstab(train.survived, train.alone)

In [ ]:
train.groupby('survived').age.describe()

In [ ]:
train.groupby('survived').fare.describe()

In [ ]:
plt.figure(figsize=(13, 7))
sns.swarmplot(data=train, y='sex', x='age', hue='survived')

# Messing around with the TELCO project dataset

In [ ]:
# We need to import data from SQL
# Connect to employees database
#defines function to create a sql url using personal credentials

from env import host, user, password

def get_db_url(database, user=user, host=host, password=password): 
    url = f'mysql+pymysql://{user}:{password}@{host}/{database}'
    return url

url = get_db_url('telco_churn')

query = '''
select * 
from customers as c
join contract_types as ct
on ct.contract_type_id = c.contract_type_id
join internet_service_types as i_s
on i_s.internet_service_type_id = c.internet_service_type_id
join payment_types as pt
on pt.payment_type_id = c.payment_type_id;
'''
df = pd.read_sql(query, url)

In [ ]:
#Cleaned Data

# Delete columns 'customer_id', contract_type_id, internet_service_type_id, payment_type_id    
df.drop(columns = ['customer_id','contract_type_id','internet_service_type_id', 'payment_type_id'], inplace = True)
# Replace partner, dependents, churn, phone_service, paperless billing, with boolean value
df.partner.replace(['Yes', 'No'], [1,0], inplace = True)
df.dependents.replace(['Yes', 'No'], [1,0], inplace = True)
df.churn.replace(['Yes', 'No'], [1,0], inplace = True)
df.phone_service.replace(['Yes', 'No'], [1,0], inplace = True)
df.paperless_billing.replace(['Yes', 'No'], [1,0], inplace = True)
# Add dummy variables as new columns in dataframe and rename them, delete origional
gender = df.gender.str.get_dummies()
df = pd.concat([df, gender], axis=1)
df.rename(columns = {'Female': 'is_female', 'Male': 'is_male'}, inplace = True)
df.drop(columns = ['gender'], inplace = True)
# Add dummy variables as new columns in dataframe and rename them, delete origional
multiple = df.multiple_lines.str.get_dummies()
df = pd.concat([df, multiple], axis=1)
df.rename(columns = {'No': 'no_multiple_lines', 'Yes': 'yes_multiple_lines'}, inplace = True)
df.drop(columns = ['multiple_lines'], inplace = True)
# Add dummy variables as new columns in dataframe and rename them, delete origional
multiple = df.online_security.str.get_dummies()
df = pd.concat([df, multiple], axis=1)
df.rename(columns = {'No': 'no_online_security', 'Yes': 'yes_online_security'}, inplace = True)
df.drop(columns = ['online_security'], inplace = True)
# Add dummy variables as new columns in dataframe and rename them, delete origional
multiple = df.online_backup.str.get_dummies()
df = pd.concat([df, multiple], axis=1)
df.rename(columns = {'No': 'no_online_backup', 'Yes': 'yes_online_backup'}, inplace = True)
df.drop(columns = ['online_backup'], inplace = True)
# Add dummy variables as new columns in dataframe and rename them, delete origional
multiple = df.device_protection.str.get_dummies()
df = pd.concat([df, multiple], axis=1)
df.rename(columns = {'No': 'no_device_protection', 'Yes': 'yes_device_protection'}, inplace = True)
df.drop(columns = ['device_protection'], inplace = True)
# Add dummy variables as new columns in dataframe and rename them, delete origional
multiple = df.tech_support.str.get_dummies()
df = pd.concat([df, multiple], axis=1)
df.rename(columns = {'No': 'no_tech_support', 'Yes': 'yes_tech_support'}, inplace = True)
df.drop(columns = ['tech_support'], inplace = True)
# Add dummy variables as new columns in dataframe and rename them, delete origional
multiple = df.streaming_tv.str.get_dummies()
df = pd.concat([df, multiple], axis=1)
df.rename(columns = {'No': 'no_streaming_tv', 'Yes': 'yes_streaming_tv'}, inplace = True)
df.drop(columns = ['streaming_tv', 'No internet service'], inplace = True)
# Add dummy variables as new columns in dataframe and rename them, delete origional
multiple = df.streaming_movies.str.get_dummies()
df = pd.concat([df, multiple], axis=1)
df.rename(columns = {'No': 'no_streaming_movies', 'Yes': 'yes_streaming_movies'}, inplace = True)
df.drop(columns = ['streaming_movies'], inplace = True)
# Add dummy variables as new columns in dataframe and rename them, delete origional
multiple = df.contract_type.str.get_dummies()
df = pd.concat([df, multiple], axis=1)
df.rename(columns = {'Month-to-month': 'month_to_month_contract', 'One year': 'one_year_contract', 'Two year': 'two_year_contract'}, inplace = True)
df.drop(columns = ['contract_type'], inplace = True)
# Add dummy variables as new columns in dataframe and rename them, delete origional
multiple = df.internet_service_type.str.get_dummies()
df = pd.concat([df, multiple], axis=1)
df.rename(columns = {'DSL': 'dsl', 'Fiber optic': 'fiber_optic'}, inplace = True)
df.drop(columns = ['internet_service_type','None'], inplace = True)
# Add dummy variables as new columns in dataframe and rename them, delete origional
multiple = df.payment_type.str.get_dummies()
df = pd.concat([df, multiple], axis=1)
df.rename(columns = {'Bank transfer (automatic)': 'auto_bank_transfer', 'Credit card (automatic)': 'auto_credit_card', 'Electronic check': 'e_check', 'Mailed check': 'mail_check'}, inplace = True)
df.drop(columns = ['payment_type'], inplace = True)
# Change total_charges to float from object
df['total_charges'] = pd.to_numeric(df['total_charges'],errors='coerce')

In [ ]:
df.info()

In [ ]:
'''
Things to Consider:
1. How to handle colums that could be booleans (Yes/No/Null)
   -Keep as object or turn to int ?
   -Columns include '''
   
  

''' contract_type
    
2. Total charges change to float, it is currently an object
3. Delete origional dummy variable columns
'''

In [ ]:
# dummy multiple lines

df